<a href="https://colab.research.google.com/github/AjeetSingh02/ExplainableAI/blob/master/SHAP_My_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install shap

In [0]:
import shap

In [0]:
X,y = shap.datasets.boston()

In [0]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [0]:
y[:5]

array([24. , 21.6, 34.7, 33.4, 36.2])

**crim**: per capita crime rate by town.<br>
**zn**: proportion of residential land zoned for lots over 25,000 sq.ft.<br>
**indus**: proportion of non-retail business acres per town.<br>
**chas**: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).<br>
**nox**: nitrogen oxides concentration (parts per 10 million).<br>
**rm**: average number of rooms per dwelling.<br>
**age**: proportion of owner-occupied units built prior to 1940.<br>
**dis**: weighted mean of distances to five Boston employment centres.<br>
**rad**: index of accessibility to radial highways.<br>
**tax**: full-value property-tax rate per 10,000. dollar<br>
**ptratio**: pupil-teacher ratio by town.<br>
**black**: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.<br>
**lstat**: lower status of the population (percent).<br>
**medv**: median value of owner-occupied homes in $1000s.

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


In [0]:
from sklearn.linear_model import LinearRegression

In [0]:
reg = LinearRegression().fit(X_train, y_train)

In [0]:
reg.score(X_test, y_test)

0.6687594935356307

In [0]:
#shap not for those processes which doesnt have predict_prob
# shap.KernelExplainer(reg, X_train, link="logit")

In [0]:
import xgboost as xgb

In [0]:
param = {
    "learning_rate": 0.01,
    } 

steps = 100  # The number of training iterations

In [0]:
#In order for XGBoost to be able to use our data, we’ll need to transform it 
#into a specific format that XGBoost can handle. That format is called DMatrix.

D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

In [0]:
model = xgb.train(param, D_train, steps)

In [0]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.DataFrame(shap_values)

In [0]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.562425,0.000973,-0.032884,0.000603,0.647425,-1.408481,0.004995,0.171888,0.000404,-0.018956,-0.067452,0.038818,-4.809988
1,0.445115,0.005917,0.036968,0.002538,0.074402,-2.004950,-0.082596,-0.115539,0.001484,0.001547,-0.538611,0.037582,1.357080
2,0.043240,0.005189,0.036888,0.000242,0.163816,-2.192333,-0.073610,-0.182980,-0.024171,0.003621,0.104001,0.049497,1.245039
3,-0.452673,0.001014,0.005627,0.000124,0.412542,-1.244130,0.011561,0.155745,0.000145,-0.019261,-0.080737,-0.364778,-4.506457
4,0.137775,0.000881,0.009788,0.002538,0.117331,-1.643562,-0.119464,-0.081326,0.001484,0.001547,-0.658268,0.061118,0.437120


In [0]:
np.average(y_train)

22.796534653465343

In [0]:
print('Expected Value:', explainer.expected_value)

Expected Value: 14.374726


In [0]:
l = 0

In [0]:
sum(df.iloc[l,:])

-6.035081216308754

In [0]:
explainer.expected_value + sum(df.iloc[l,:])

8.339645079162437

In [0]:
shap.initjs()

shap.force_plot(explainer.expected_value, shap_values[l,:], X_train.iloc[l,:])

In [0]:
X_train.iloc[l,:]

CRIM        15.0234
ZN           0.0000
INDUS       18.1000
CHAS         0.0000
NOX          0.6140
RM           5.3040
AGE         97.3000
DIS          2.1007
RAD         24.0000
TAX        666.0000
PTRATIO     20.2000
B          349.4800
LSTAT       24.9100
Name: 477, dtype: float64

In [0]:
y_train[l]

12.0

SHAP (SHapley Additive exPlanations) assigns each feature an importance value for a particular prediction. Its novel components include: the identification of a new class of additive feature importance measures, and theoretical results showing there is a unique solution in this class with a set of desirable properties. Typically, SHAP values try to explain the output of a model (function) as a sum of the effects of each feature being introduced into a conditional expectation. Importantly, for non-linear functions the order in which features are introduced matters. The SHAP values result from averaging over all possible orderings. Proofs from game theory show this is the only possible consistent approach.

**Deepanjans_notebook**

In [0]:
import pandas as pd
import numpy as np
# import model_evaluation_utils as meu
import matplotlib.pyplot as plt
from collections import Counter
import shap
# import eli5

import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
%matplotlib inline

shap.initjs()

In [0]:
data, labels = shap.datasets.adult(display=True)
labels = np.array([int(label) for label in labels])
data.shape, labels.shape

((32561, 12), (32561,))

The class label column is the one we want to predict
(False: Income <= \$50K & True: Income > \$50K)

In [0]:
data.head()

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
0,39.0,State-gov,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States
1,50.0,Self-emp-not-inc,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States
2,38.0,Private,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States
3,53.0,Private,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States
4,28.0,Private,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba


In [0]:
#typically the XGBoost model can handle categorical data natively being a tree-based model so we don't one-hot encode the features

cat_cols = data.select_dtypes(['category']).columns
data[cat_cols] = data[cat_cols].apply(lambda x: x.cat.codes)
data.head()

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
0,39.0,7,13.0,4,1,1,4,1,2174.0,0.0,40.0,39
1,50.0,6,13.0,2,4,0,4,1,0.0,0.0,13.0,39
2,38.0,4,9.0,0,6,1,4,1,0.0,0.0,40.0,39
3,53.0,4,7.0,2,6,0,2,1,0.0,0.0,40.0,39
4,28.0,4,13.0,2,10,5,2,0,0.0,0.0,40.0,5


In [0]:
Counter(labels)

Counter({0: 24720, 1: 7841})

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
X_train.shape, X_test.shape


((22792, 12), (9769, 12))

In [0]:
X_train.head(3)

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
19749,34.0,6,9.0,2,5,5,4,0,0.0,2179.0,12.0,39
1216,48.0,6,10.0,2,3,0,0,1,7688.0,0.0,40.0,39
27962,23.0,7,10.0,2,10,0,4,1,0.0,0.0,30.0,39


In [0]:
data_disp, labels_disp = shap.datasets.adult(display=True)
X_train_disp, X_test_disp, y_train_disp, y_test_disp = train_test_split(data_disp, labels_disp, test_size=0.3, random_state=42)
X_train_disp.shape, X_test_disp.shape

((22792, 12), (9769, 12))

In [0]:
X_train_disp.head(3)

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
19749,34.0,Self-emp-not-inc,9.0,Married-civ-spouse,Farming-fishing,Wife,White,Female,0.0,2179.0,12.0,United-States
1216,48.0,Self-emp-not-inc,10.0,Married-civ-spouse,Craft-repair,Husband,Amer-Indian-Eskimo,Male,7688.0,0.0,40.0,United-States
27962,23.0,State-gov,10.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,30.0,United-States


In [0]:
columns = X_train_disp.columns

In [0]:
%%time

import xgboost as xgb

xgc = xgb.XGBClassifier(n_estimators=500, max_depth=5, base_score=0.5,
                        objective='binary:logistic', random_state=42)
xgc.fit(X_train, y_train)

CPU times: user 6.76 s, sys: 152 ms, total: 6.91 s
Wall time: 7.67 s


In [0]:
predictions = xgc.predict(X_test)
predictions[:10]

array([0, 0, 1, 0, 0, 1, 1, 0, 0, 1])

In [0]:
np.average(y_train)

0.2424973674973675

In [0]:
explainer = shap.TreeExplainer(xgc)
shap_values = explainer.shap_values(X_test)

In [0]:
explainer.__dict__

{'data': None,
 'data_missing': None,
 'expected_value': -1.3625857,
 'feature_perturbation': 'tree_path_dependent',
 'model': <shap.explainers.tree.TreeEnsemble at 0x7f7006e49710>,
 'model_output': 'margin'}

In [0]:
shap_df = pd.DataFrame(shap_values, columns = columns)

In [0]:
shap_df.head()

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
0,-1.209206,-0.003895,-0.073927,-1.078541,-0.645259,-0.517731,-0.002871,-0.421329,-0.144742,-0.036394,-0.568053,-0.004587
1,0.354368,-0.262398,-0.483883,0.586233,0.307967,1.194750,0.017408,-0.216272,-0.108322,-0.048158,-0.095782,0.017657
2,-0.525650,0.030637,0.970991,0.360480,0.528695,0.603971,-0.103895,0.050630,-0.176231,-0.087669,0.509656,0.024573
3,-0.426094,0.076188,0.576406,-1.108961,-0.517884,-0.522604,0.028896,-0.359869,-0.142805,-0.057430,-0.250737,0.023183
4,-0.503867,-0.159754,-0.129339,-0.547773,-0.049194,-0.414065,-0.019701,0.128197,-3.685364,-0.031033,0.781617,0.001285


In [0]:
X_test_disp.head()

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
14160,27.0,Private,10.0,Divorced,Adm-clerical,Not-in-family,White,Female,0.0,0.0,38.0,United-States
27048,45.0,State-gov,9.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States
28868,29.0,Private,13.0,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0.0,0.0,55.0,United-States
5667,30.0,Private,13.0,Never-married,Machine-op-inspct,Not-in-family,White,Female,0.0,0.0,40.0,United-States
7827,29.0,Self-emp-not-inc,10.0,Divorced,Craft-repair,Not-in-family,White,Male,2202.0,0.0,50.0,United-States


In [0]:
l = 50

In [0]:
print('Expected Value:', explainer.expected_value)
print("sum of values in a row in shap_df: ", sum(shap_df.iloc[l,:]))
print("output value: ", explainer.expected_value + sum(shap_df.iloc[l,:]))

Expected Value: -1.3625857
sum of values in a row in shap_df:  -2.5777042992413044
output value:  -3.9402899630367756


In [0]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[l,:], X_test_disp.iloc[l,:])

In [0]:
shap.initjs()

shap.force_plot(explainer.expected_value, shap_values[:l,:], X_test_disp.iloc[:l,:])

**Attribute Information**:<br>
**age**: continuous.<br>
**workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.<br>
**education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.<br>
**marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.<br>
**occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.<br>
**relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.<br>
**race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.<br>
**sex**: Female, Male.<br>
**capital-gain**: continuous.<br>
**capital-loss**: continuous.<br>
**hours-per-week**: continuous.<br>
**native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.<br>

**class**: >50K, <=50K

In [0]:
import xgboost
import shap

# load JS visualization code to notebook
shap.initjs()

# train XGBoost model
X ,y = shap.datasets.boston()
model = xgboost.train({"learning_rate": 0.01}, xgboost.DMatrix(X, label=y), 100)

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

In [0]:
shap.initjs()

l = 0

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[l,:], X.iloc[l,:])

In [0]:
shap_df = pd.DataFrame(shap_values, columns = X.columns)

In [0]:
X.head(1)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98


In [0]:
shap_df.head(1)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.214554,0.001347,0.018625,0.0,-0.407199,-1.161835,-0.051294,-0.342014,-0.040446,0.005487,0.050437,0.027335,3.944318


In [0]:
shap_df["RM"]

0     -1.161835
1     -1.421002
2      2.662229
3      2.100868
4      2.622416
         ...   
501   -0.255168
502   -1.770272
503    1.608566
504   -0.260753
505   -2.511635
Name: RM, Length: 506, dtype: float32

In [0]:
for col in shap_df.columns:
    print(col, sum(shap_df.loc[:,col]))

CRIM -6.468349045142531
ZN -0.5351447672546783
INDUS 4.376913574989885
CHAS 0.0
NOX 20.356929237022996
RM -100.27523138746619
AGE -10.692283506505191
DIS 42.78981123527046
RAD 0.38649408938363194
TAX 9.21235652444011
PTRATIO 1.6863267119624652
B 0.3805617548059672
LSTAT 38.7816094760783


In [0]:
Overall, DIS has the highest affect on swaying the model output higher than average. 

RM  has the highest affect on swaying the model output lower than average. 